In [1]:
import pandas as pd
import numpy as np

import time
import pickle

In [3]:
matrix = pd.read_pickle('D:/ml_data/data_lstm_full.pkl')

In [4]:
from sklearn.preprocessing import MinMaxScaler

columns = [column for column in matrix.columns if column not in ['date_block_num', 'shop_id', 'item_id', 'item_cnt_month']]

matrix.replace([np.inf, -np.inf], np.nan, inplace=True)
matrix.fillna(0, inplace=True)

scaler_X = MinMaxScaler()
matrix[columns] = scaler_X.fit_transform(matrix[columns].values)

# scaler_y = MinMaxScaler()
# matrix['item_cnt_month'] = scaler_y.fit_transform(matrix['item_cnt_month'].values.reshape(-1, 1))

In [5]:
from sklearn.externals import joblib

joblib.dump(scaler_X, 'scaler_X_2.pkl')
# joblib.dump(scaler_y, 'scaler_y.pkl')

C:\Users\HWer\Anaconda3\envs\ml_project\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['scaler_X_2.pkl']

In [6]:
matrix.to_pickle('D:/ml_data/data_lstm_X_scaled.pkl')

In [2]:
matrix = pd.read_pickle('D:/ml_data/data_lstm_X_scaled.pkl')

In [26]:
def window_stack(a, stepsize=1, width=2):
    n = a.shape[1]
    omg = lambda i: 1+n+i-width
    _X = [a[:, i:i+width].astype(np.float16) for i in range(0, n - width)]
    _y = [a[:, i].astype(np.float16) for i in range(width, n)]
    del a
    _X = np.stack(_X, axis=1).astype(np.float16)
    _y = np.stack(_y, axis=1).astype(np.float16)

    return _X.reshape(-1, _X.shape[2], _X.shape[3]), _y.reshape(-1, _y.shape[2])

In [4]:
def get_X_y(data, width=7):
    data.fillna(0, inplace=True)
    
    test_data = data[(data.date_block_num >= (33 - width)) & (data.date_block_num < 34)]
    X = np.array(list(test_data.groupby(['shop_id', 'item_id']).apply(pd.DataFrame.as_matrix)))
    del test_data
    
    X, y = window_stack(X, width=width)
    test_X = X[:, :, 3:]
    test_y = y[:, 3]
    del X
    del y
    
    train_data = data[data.date_block_num < 33]
    del data
    
    X = np.array(list(train_data.groupby(['shop_id', 'item_id']).apply(pd.DataFrame.as_matrix)))
    del train_data
    
    X, y = window_stack(X, width=width)
    train_X = X[:, :, 3:]
    train_y = y[:, 3]
    del X
    del y
        
    return train_X, train_y, test_X, test_y

In [5]:
widths = [7, 14, 21, 28]

In [6]:
for width in widths:
    ts = time.time()
    train_X, train_y, test_X, test_y = get_X_y(matrix, width)
    
    train_data_file = 'train_data_Xscaled_{}.npz'.format(width)
    test_data_file = 'test_data_Xscaled_{}.npz'.format(width)
    
    np.savez_compressed(train_data_file, X=train_X, y=train_y)
    np.savez_compressed(test_data_file, X=test_X, y=test_y)
    
    print(train_X.dtype)
    del train_X
    del train_y
    del test_X
    del test_y
    
    print(time.time() - ts)

C:\Users\HWer\Anaconda3\envs\ml_project\lib\site-packages\pandas\core\groupby\ops.py:942: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return reduction.apply_frame_axis0(sdata, f, names, starts, ends)


float16
214.28668212890625
float16
321.5605342388153
float16
412.7312242984772


In [7]:
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

NameError: name 'train_X' is not defined

In [1]:
from keras.optimizers import SGD, Adagrad, Adadelta, RMSprop, Adam, Adamax, Nadam
# from Eve.Eve import Eve


__name_to_optimizer = {
    'rmsprop': RMSprop,
    'adam': Adam,
    'sgd': SGD,
    'adagrad': Adagrad,
    'adadelta': Adadelta,
    'adamax': Adamax,
    'nadam': Nadam,
    # 'eve': Eve,
}


def make_optimizer(optimizer_name='adam', **optimizer_params):
    try:
        optimizer = __name_to_optimizer[optimizer_name]
    except KeyError as e:
        raise ValueError('Undefined optimizer: {}'.format(e.args[0]))

    return optimizer(**optimizer_params)

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
import keras.backend as K

In [3]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [4]:
from keras.regularizers import l1_l2


__name_to_loss = {
    'mse': 'mse',
    'rmse': root_mean_squared_error,
}

def make_loss(loss_name):
    return __name_to_loss[loss_name]

    
class LSTMWrapper(object):
    def __init__(self, features_count, units, dropout_rate, regularization_param, epochs_count, loss, optimizer):
        self.features_count = features_count
        self.units = units
#         self.activation = activation
        self.dropout_rate = dropout_rate
        self.regularizer = l1_l2(l1=regularization_param, l2=regularization_param)
        self.epochs_count = epochs_count
        self.loss = make_loss(loss)
        self.optimizer_name = optimizer
        self.initializer = 'glorot_normal'
        self.learning_rate=0.0001
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()

        lstm_layer = LSTM(
            units=self.units,
            input_shape=(self.features_count, 75),
            kernel_regularizer=self.regularizer,
            kernel_initializer=self.initializer,
        )
        
        model.add(lstm_layer)
#         model.add(BatchNormalization())
        model.add(Dropout(self.dropout_rate))

        model.add(Dense(1))

        opt = make_optimizer(self.optimizer_name, lr=self.learning_rate)
        model.compile(loss=self.loss, optimizer=opt, metrics=[root_mean_squared_error])

        return model

    def score_sklearn(self, test_data, metrics, scaler=None):
        if metrics is str:
            metrics = [metrics]

        y_preds = self.model.predict(test_data[0])
        ys = test_data[1]

        scores = dict()
        for metric_name in metrics:
            scores['predictor_{}'.format(metric_name)] = make_sklearn_metric(metric_name)(ys, y_preds)

        return scores


model_params = dict(
    features_count = 33,
    units=64,
    dropout_rate=0.5,
    regularization_param=1e-3,
    epochs_count=200,
    loss='rmse',
    optimizer='adam',
)
lstm = LSTMWrapper(**model_params)
# lstm.model.fit(X_train, y_train, batch_size = 4096, epochs = 1, validation_data=(X_valid, y_valid))

In [2]:
import pandas as pd
import numpy as np
import time

In [6]:
matrix = pd.read_pickle('D:/ml_data/data_lstm_X_scaled.pkl')

In [7]:
from sklearn.model_selection import ParameterGrid, ParameterSampler

param_test = dict(
    features_count=[33], #[7, 14, 21, 28]
    units=[16, 32, 64, 128],
    dropout_rate=[0.1, 0.25, 0.5],
    regularization_param=[10 ** -i for i in range(3, 7)],
    epochs_count=[100],
    loss=['mse', 'rmse'],
    optimizer=['adam', 'rmsprop'],
)

sampler = ParameterSampler(param_test, n_iter=25, random_state=123)

In [4]:
import os

In [5]:
trained_models_dir = 'trained_models_dir'
trained_models_dir = trained_models_dir + '/lstm'
if not os.path.exists(trained_models_dir):
    os.makedirs(trained_models_dir)

In [10]:
def get_inner_dirs(path):
    for file in os.listdir(path):
        if os.path.isdir(os.path.join(path, file)):
            yield file

In [11]:
def __create_model_directory(models_dir) -> str:
    inner_dirs = list(get_inner_dirs(models_dir))
    folder_name = 'gs_{}'.format(len(inner_dirs))

    model_dir = os.path.join(models_dir, folder_name)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    return model_dir, folder_name

In [12]:
def get_X_y(data):
    #     data = data[data.date_block_num > 11]
    X = data.drop(['date_block_num', 'item_cnt_month', 'shop_id', 'item_id'], axis=1)
    y = data['item_cnt_month']
    return X, y

In [8]:
import json

In [14]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [15]:
import numpy as np

In [6]:
search_results = []
search_results_file = os.path.join(trained_models_dir, 'search_results.json')

In [16]:
def get_X_y_train(data):
    X = data[data.date_block_num < 33]
    X = np.array(list(X.groupby(['shop_id', 'item_id']).apply(pd.DataFrame.as_matrix)))
    X = X[:, :, 3:]
    y = data[data.date_block_num == 33]['item_cnt_month']
    
    return X, y

X, y = get_X_y_train(matrix)

C:\Users\HWer\Anaconda3\envs\ml_project\lib\site-packages\pandas\core\groupby\ops.py:942: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return reduction.apply_frame_axis0(sdata, f, names, starts, ends)


In [18]:
del matrix

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=123)

In [20]:
del X
del y

In [21]:
for params in list(sampler):
        print('current parameters : {}'.format(params))

        model_dir, folder_name = __create_model_directory(trained_models_dir)
        print('model directory : {}'.format(model_dir))
        lstm = LSTMWrapper(**params)
        
#         width = params['features_count']
#         train_data_file = 'train_data_Xscaled_{}.npz'.format(width)
#         valid_data_file = 'test_data_Xscaled_{}.npz'.format(width)
        
#         train_data = np.load(train_data_file)
#         X_train = train_data['X'].astype(np.float16)
#         y_train = train_data['y'].astype(np.float16)
#         if X_train.shape[0] > 1200000:
#             n = 1200000
#             index = np.random.choice(X.shape[0], n, replace=False)  
#             X_train = X_train[index]
#             y_train = y_train[index]
#         del train_data
        
#         valid_data = np.load(valid_data_file)
#         X_valid = valid_data['X'].astype(np.float16)
#         y_valid = valid_data['y'].astype(np.float16)
#         del valid_data
        
        lstm.model.fit(X_train, y_train, batch_size = 65536, epochs = 5, validation_data=(X_valid, y_valid))
        lstm.model.save(os.path.join(model_dir, 'lstm.h5'))
        
        
        mean_cv_scores = [
            rmse(y_train, lstm.model.predict(X_train)),
            rmse(y_valid, lstm.model.predict(X_valid)),
#             rmse(test_y, lstm.model.predict(test_X)),
        ]
        
        write_message = dict(
            params=params,
            scores=dict(
                train=mean_cv_scores[0],
                val=mean_cv_scores[1],
#                 test=mean_cv_scores[2],
            ),
            model_path=model_dir,
        )

        search_results.append(write_message)

        with open(search_results_file, 'w') as file:
            json.dump(search_results, file)
            print('overwrite model parameters file ({})'.format(search_results_file))
            
#         del X_train
#         del y_train
        
#         del X_valid
#         del y_valid

current parameters : {'units': 16, 'regularization_param': 1e-06, 'optimizer': 'adam', 'loss': 'rmse', 'features_count': 33, 'epochs_count': 100, 'dropout_rate': 0.5}
model directory : trained_models_dir/lstm\gs_7
Train on 171360 samples, validate on 42840 samples
Epoch 1/5
171360/171360 [==============================] - 76s 445us/step - loss: 2.6075 - root_mean_squared_error: 2.7586 - val_loss: 2.6529 - val_root_mean_squared_error: 2.6524
Epoch 2/5
171360/171360 [==============================] - 44s 256us/step - loss: 2.5233 - root_mean_squared_error: 2.7065 - val_loss: 2.6522 - val_root_mean_squared_error: 2.6516
Epoch 3/5
171360/171360 [==============================] - 17s 98us/step - loss: 2.6075 - root_mean_squared_error: 2.6798 - val_loss: 2.6515 - val_root_mean_squared_error: 2.6509
Epoch 4/5
171360/171360 [==============================] - 12s 67us/step - loss: 2.4995 - root_mean_squared_error: 2.3800 - val_loss: 2.6508 - val_root_mean_squared_error: 2.6502
Epoch 5/5
171360/

KeyboardInterrupt: 

In [9]:
with open(search_results_file, 'r') as file:
    search_results = json.load(file)

In [10]:
val_scores = [record['scores']['val'] for record in search_results]
min_index = val_scores.index(min(val_scores))

In [11]:
best_model = search_results[min_index]
best_model

{'params': {'units': 16,
  'regularization_param': 1e-06,
  'optimizer': 'adam',
  'loss': 'rmse',
  'features_count': 33,
  'epochs_count': 100,
  'dropout_rate': 0.5},
 'scores': {'train': 2.67440765097046, 'val': 2.6495462532966196},
 'model_path': 'trained_models_dir/lstm\\gs_7'}

In [ ]:
from keras.models import load_model

model = load_model(os.path.join(best_model['model_path'], 'lstm.h5'), custom_objects={'root_mean_squared_error': root_mean_squared_error})

In [ ]:
width = best_model['params']['features_count']

In [20]:
train_data_file = 'train_data_Xscaled_{}.npz'.format(width)
valid_data_file = 'test_data_Xscaled_{}.npz'.format(width)

train_data = np.load(train_data_file)
X_train = train_data['X'].astype(np.float16)
y_train = train_data['y'].astype(np.float16)
del train_data

valid_data = np.load(valid_data_file)
X_valid = valid_data['X'].astype(np.float16)
y_valid = valid_data['y'].astype(np.float16)
del valid_data

In [21]:
train_pred = model.predict(X_train)
valid_pred = model.predict(X_valid)

# y_train = scaler_y.inverse_transform(y_train.reshape(-1, 1)).clip(0, 20)
# y_valid = scaler_y.inverse_transform(y_valid.reshape(-1, 1)).clip(0, 20)

# train_pred = scaler_y.inverse_transform(train_pred.reshape(-1, 1)).clip(0, 20)
# valid_pred = scaler_y.inverse_transform(valid_pred.reshape(-1, 1)).clip(0, 20)

mean_cv_scores = [
    rmse(y_train, train_pred),
    rmse(y_valid, valid_pred),
#   rmse(test_y, lstm.model.predict(test_X)),
]
mean_cv_scores

[2.7020194378202387, 2.7225219168012917]

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
matrix = pd.read_pickle('D:/ml_data/data_lstm_X_scaled.pkl')

In [ ]:
def get_X_y_test(data, width=7):
#     data.fillna(0, inplace=True)
    
#     test_data = data[(data.date_block_num >= (34 - width)) & (data.date_block_num < 35)]
#     X = np.array(list(test_data.groupby(['shop_id', 'item_id']).apply(pd.DataFrame.as_matrix)))
#     del test_data
    
#     X, y = window_stack(X, width=width)
#     test_X = X[:, :, 3:]
#     test_y = y[:, 1:4]
#     del X
#     del y
    X = data[(data.date_block_num > 0) & (data.date_block_num < 34)]
    X = np.array(list(X.groupby(['shop_id', 'item_id']).apply(pd.DataFrame.as_matrix)))
    X = X[:, :, 3:]
    y = data[data.date_block_num == 34][['shop_id', 'item_id', 'item_cnt_month']]
    
    return X, y

In [ ]:
import time


ts = time.time()
test_X, test_y = get_X_y_test(matrix, width)

# train_data_file = 'train_data_Xscaled_{}.npz'.format(width)
# test_data_file = 'test_data_Xscaled_{}.npz'.format(width)

# np.savez_compressed(train_data_file, X=train_X, y=train_y)
# np.savez_compressed(test_data_file, X=test_X, y=test_y)

# del test_X
# del test_y

print(time.time() - ts)

In [42]:
test_data_file

'test_data_Xscaled_28.npz'

In [ ]:
test_data = np.load(test_data_file)

In [ ]:
test_X = test_data['X']
test_y = test_data['y']

In [29]:
test_y_df = pd.DataFrame(data=test_y, columns=['shop_id', 'item_id', 'item_cnt_month'])

In [ ]:
from module.data.read_data import *

test = test_file_processing().set_index('ID')
test.shape

In [ ]:
test = test[['shop_id', 'item_id']]

In [ ]:
test.head()

In [ ]:
_pred = model.predict(test_X)
test_y['item_cnt_month'] = _pred.clip(0, 20)

In [ ]:
test_y.shape

In [ ]:
_pred.shape

In [ ]:
answer = pd.merge(test, test_y,  how='left', on=['shop_id', 'item_id'])

In [ ]:
answer.head()

In [ ]:
answer.shape

In [ ]:
submission = pd.DataFrame({
    "ID": answer.index, 
    "item_cnt_month": answer['item_cnt_month']
})

submission.fillna(0, inplace=True)
# submission['ID'] = submission['ID'].astype(np.int32)
submission['item_cnt_month'] = submission['item_cnt_month']

In [ ]:
import pickle

pickle.dump(_pred, open('lstm_test.pickle', 'wb'))

In [ ]:
submission.head()

In [ ]:
submission.min()

In [ ]:
submission.max()

In [ ]:
submission.to_csv('lstm_submission.csv', index=False)

In [ ]:
submission.info()